In [ ]:
import glob
import torch
import pickle
import numpy as np


In [ ]:
obs_labels = []
for filename in sorted(glob.glob('/AS_data/obs_npy/*.npy')):
    print(filename+'   is loading')
    obs_label = np.load(filename)
    tick,_,W,H = obs_label.shape
    obs_labels.append(obs_label[:,2].copy())
    del obs_label

fine_inputs = []
for filename in sorted(glob.glob('/AS_data/finetune_input_npy/*.npy')):
    print(filename+'   is loading')
    EM = np.load(filename)
    tick,_,W,H = EM.shape
    print(tick)
    fine_inputs.append(EM[:].copy())

org_inputs = []
for filename in sorted(glob.glob('/AS_data/Emis_npy/*.npy')):
    print(filename+'   is loading')
    EM = np.load(filename)
    tick,_,W,H = EM.shape
    print(tick)
    org_inputs.append(EM[:].copy())

In [ ]:
for i in range(1): #len(fine_inputs)):
    obs_label = obs_labels[i].copy()
    obs_label[obs_label>0] = 1
    obs_label[obs_label<=0] = 0
    obs_label = np.mean(obs_label,axis = 0)
    plt.imshow(obs_label[::-1],cmap=plt.cm.hot_r)
    plt.title('observation')
    plt.colorbar()
    plt.show()
    
    for em_idx in range(51): #fine_inputs[0].shape[1]):
        fine_input = fine_inputs[i][:,em_idx].copy()
        org_input = org_inputs[i][:,em_idx].copy()
        
        fine_input = np.sum(fine_input,axis = 0)
        org_input = np.sum(org_input,axis = 0)
#         org_input[org_input == 0] = 1e-1
        
        a = fine_input/(org_input+1e-1)
#         a[a>100] = 100
#         a[a<-100] = -100
        
#         a[obs_label<=0.5] = np.min(a)
        
        plt.imshow(a[::-1],cmap=plt.cm.hot_r)
        plt.title(dic[em_idx])
        plt.colorbar()
        plt.show()
        
        
        a = fine_input - org_input
        plt.imshow(a[::-1],cmap=plt.cm.hot_r)
        plt.title('difference')
        plt.colorbar()
        plt.show()
        
        plt.figure(figsize=(15,5))
        plt.imshow(np.concatenate([fine_input[::-1],org_input[::-1]],axis = 1),cmap=plt.cm.hot_r)
        plt.title(dic[em_idx])
        plt.colorbar()
        plt.show()
        
        a = a[obs_label>0.5]
        print(np.mean(np.abs(a[a!=0])))
        plt.hist(a[a!=0].reshape(-1),bins = 100)
        plt.title(dic[em_idx])
        plt.show()